In [1]:
import pandas as pd
from pandas_datareader import data as pdr
import FinanceDataReader as fdr
import yfinance
from tqdm import tqdm
import time
import warnings
import talib
warnings.filterwarnings("ignore")

In [2]:
def get_position(df):
    close = df["Adj Close"]

    # 볼린저밴드
    df["Upper"], df["Middle"], df["Lower"] = talib.BBANDS(close, timeperiod=20)

    # 모멘텀
    df["Momentum"] = talib.MOM(close, timeperiod=10)
    df["Momentum_signal"] = talib.SMA(df["Momentum"], timeperiod=9)

    # 이평선
    df["MA5"] = talib.SMA(close, timeperiod=5)
    df["MA15"] = talib.SMA(close, timeperiod=20)
    df["MA20"] = talib.SMA(close, timeperiod=20)
    df["MA60"] = talib.SMA(close, timeperiod=60)

    # RSI
    df["RSI"] = talib.RSI(close, timeperiod=14)
    df["RSI_signal"] = talib.SMA(df["RSI"], timeperiod=6)

    # MACD
    df["MACD"], df["MACD_signal"], df["MACD_hist"] = talib.MACD(close)

    return df

tickers = ["ADBE", "XLV", "WDC", "QCOM", "MDLZ", "IAU", "USOI"]

for ticker in tickers:
    time.sleep(1)
    df = pdr.get_data_yahoo(ticker, "2022")
    df = get_position(df)
    df["Score"] = 0
    temp_dict = dict(df)
    sell_threshold = -4

    # 주가가 밴드 상단보다 높으면 -1
    if temp_dict["Adj Close"][-1] > temp_dict["Upper"][-1]:
        temp_dict["Score"][-1] -= 2
    # 주가가 밴드 하단보다 낮으면 +1
    elif temp_dict["Adj Close"][-1] < temp_dict["Lower"][-1]:
        temp_dict["Score"][-1] += 2

    # 모멘텀이 0보다 위일 때 +1
    if temp_dict["Momentum"][-1] > 0:
        temp_dict["Score"][-1] += 1
    # 모멘텀이 0보다 아래일 때 -1
    elif temp_dict["Momentum"][-1] < 0:
        temp_dict["Score"][-1] -= 1

    # 모멘텀이 모멘텀 시그널을 상향돌파시 +1
    if (temp_dict["Momentum"][-2] < temp_dict["Momentum_signal"][-2]) and (temp_dict["Momentum_signal"][-1] < temp_dict["Momentum"][-1]):
        temp_dict["Score"][-1] += 2
    # 모멘텀이 모멘텀 시그널을 하향돌파시 -1
    elif (temp_dict["Momentum"][-2] > temp_dict["Momentum_signal"][-2]) and (temp_dict["Momentum_signal"][-1] > temp_dict["Momentum"][-1]):
        temp_dict["Score"][-1] -= 2

    # 단기 이평선이 장기 이평선을 상향돌파시 +1
    if (temp_dict["MA15"][-2] < temp_dict["MA20"][-2]) and (temp_dict["MA20"][-1] < temp_dict["MA15"][-1]):
        temp_dict["Score"][-1] += 1
    # 단기 이평선이 장기 이평선을 하향돌파시 -1
    elif (temp_dict["MA15"][-2] > temp_dict["MA20"][-2]) and (temp_dict["MA20"][-1] > temp_dict["MA15"][-1]):
        temp_dict["Score"][-1] -= 1

    # RSI가 70을 넘어가면 과매수 -1
    if temp_dict["RSI"][-1] >= 70:
        temp_dict["Score"][-1] -= 2
    # RSI가 30에서 내려가면 과매도 +1
    elif temp_dict["RSI"][-1] <= 30:
        temp_dict["Score"][-1] += 2

    # RSI가 RSI 시그널을 상향돌파시 +1
    if (temp_dict["RSI"][-2] < temp_dict["RSI_signal"][-2]) and (temp_dict["RSI_signal"][-1] < temp_dict["RSI"][-1]):
        temp_dict["Score"][-1] += 1
    # RSI가 RSI 시그널을 하향돌파시 -1
    elif (temp_dict["RSI"][-2] > temp_dict["RSI_signal"][-2]) and (temp_dict["RSI_signal"][-1] > temp_dict["RSI"][-1]):
        temp_dict["Score"][-1] -= 1

    # MACD가 MACD 시그널을 상향돌파시 +1
    if (temp_dict["MACD"][-2] < temp_dict["MACD_signal"][-2]) and (temp_dict["MACD_signal"][-1] < temp_dict["MACD"][-1]):
        temp_dict["Score"][-1] += 1
    # MACD가 MACD 시그널을 하향돌파시 -1
    elif (temp_dict["MACD"][-2] > temp_dict["MACD_signal"][-2]) and (temp_dict["MACD_signal"][-1] > temp_dict["MACD"][-1]):
        temp_dict["Score"][-1] -= 1

    df = pd.DataFrame.from_dict(temp_dict)
    #USD = 1303.97
    print(ticker)
    print("현재 가격 :", df.iloc[-1]["Adj Close"])
    print("매도 스코어 :", df.iloc[-1]["Score"])
    is_sell = True if df.iloc[-1]["Score"] <= sell_threshold else False
    print("매도 여부 :", is_sell)
    print("")

ADBE
현재 가격 : 437.82000732421875
매도 스코어 : -2.0
매도 여부 : False

XLV
현재 가격 : 133.14999389648438
매도 스코어 : -2.0
매도 여부 : False

WDC
현재 가격 : 48.20000076293945
매도 스코어 : -4.0
매도 여부 : True

QCOM
현재 가격 : 148.52999877929688
매도 스코어 : -2.0
매도 여부 : False

MDLZ
현재 가격 : 65.51000213623047
매도 스코어 : -1.0
매도 여부 : False

IAU
현재 가격 : 33.5
매도 스코어 : -1.0
매도 여부 : False

USOI
현재 가격 : 4.550000190734863
매도 스코어 : 1.0
매도 여부 : False



In [3]:
df.tail()

,High,Low,Open,Close,Volume,Adj Close,Upper,Middle,Lower,Momentum,...,MA5,MA15,MA20,MA60,RSI,RSI_signal,MACD,MACD_signal,MACD_hist,Score
Date,,,,,,,,,,,,,,,,,,,,,
2022-08-11,4.780,4.68,4.72,4.74,2027500.0,4.74,4.901685,4.690776,4.479868,-0.07,...,4.616,4.690776,4.690776,4.953147,50.597137,43.732718,-0.060832,-0.072713,0.011881,0
2022-08-12,4.720,4.67,4.72,4.70,1746500.0,4.70,4.902340,4.696000,4.489660,-0.17,...,4.660,4.696000,4.696000,4.946434,48.822625,45.458865,-0.053244,-0.068819,0.015576,0
2022-08-15,4.629,4.50,4.52,4.61,3359000.0,4.61,4.897029,4.689000,4.480971,-0.09,...,4.660,4.689000,4.689000,4.939400,44.998620,46.599549,-0.053871,-0.065830,0.011959,0
2022-08-16,4.650,4.50,4.61,4.53,3332100.0,4.53,4.889579,4.676000,4.462421,-0.23,...,4.648,4.676000,4.676000,4.929836,41.860060,46.168332,-0.060131,-0.064690,0.004559,0
2022-08-17,4.600,4.51,4.54,4.55,2862900.0,4.55,4.877579,4.664000,4.450421,-0.06,...,4.626,4.664000,4.664000,4.920297,42.931705,46.038535,-0.062754,-0.064303,0.001549,1
